$\bf{Problem Statement}$: Write a function that takes (ra,dec,distance,proper motion ra,proper motion dec,line-of-sight velocity) and their Gaussian uncertainties for a star and computes eccentricity, zmax, rperi, rap and their uncertainties using Monte Carlo sampling with the galpy function galpy.actionAngle.actionAngleStaeckel.EccZmaxRperiRap. Apply to some stars in the Gaia RV sample.

In [216]:
import numpy as np
from galpy.actionAngle import actionAngleStaeckel
from galpy.potential import MWPotential2014
from astropy.coordinates import ICRS, Galactocentric
import astropy.units as u

In [217]:
# create a action angle Staeckle object using the approximation delta = 0.4 and runs on c that makes the code faster
aAS = actionAngleStaeckel(pot=MWPotential2014,delta=0.4,c=True)

Create a function that convert rad dec coordinate to galactocentric coordinate

In [218]:
def ICRS_to_Galactocentric_Cylindrical(ra, dec, distance, pm_ra, pm_dec, vr):
    """
    Take a star's coordinate in ICRS form and return the galactocentric cylindrical coordinate in a tuple.
    
    Args:
        ra (float): the right ascension in degree
        dec (float): the declination in degree
        distance (float): the distance in kpc
        pm_ra (float): the proper motion in the right ascension direction in milliarcsec per year
        pm_dec (float): the proper motion in the declination direction in milliarcsec per year
        radial_velocity (float): the radial velocity in kilometer per second
        
    Returns:
        (R, phi, z, vR, vT, vz): a tuple of 6 Quantity representing the galactocentric cylindrical coordinates
    """
    # create an astroy coordinate object that stores the star's coordinate in ICRS form
    star = coord.ICRS(ra = ra*u.degree, dec = dec*u.degree, distance = distance*u.kpc, pm_ra_cosdec = pm_ra*u.mas/u.yr,
              pm_dec = pm_dec*u.mas/u.yr, radial_velocity = vr*u.km/u.s)
    # convert it to galactocentric coordinate
    star = star.transform_to(Galactocentric)
    # change it to cylindrical polar coordinate
    star.representation = 'cylindrical'
    # get the cylindrical polar coordinate attributes
    R, phi, z, vR, d_phi, vz  = star.rho, star.phi, star.z, star.d_rho, star.d_phi, star.d_z
    # change angular speed to unit of radian and take out the radian unit
    d_phi.to(u.rad/u.yr)
    d_phi = d_phi/u.rad
    # convert angular speed to tangential speed
    vT = d_phi * R
    # return the attributes
    return (R, phi, z, vR, vT, vz)

Create a function that computes eccentricity, zmax, rperi, rap

In [219]:
def find_EccZmaxRperiRap(ra, dec, distance, pm_ra, pm_dec, vr):
    """
    Args:
        ra (float): the right ascension in degree
        dec (float): the declination in degree
        distance (float): the distance in kpc
        pm_ra (float): the proper motion in the right ascension direction in milliarcsec per year
        pm_dec (float): the proper motion in the declination direction in milliarcsec per year
        radial_velocity (float): the radial velocity in kilometer per second
        
    Returns:
        Ecc (float): the eccentricity of the orbit 
        Zmax (float): the maximum height of the orbit
        Rperi (float): the pericenter
        Rap (float): the apocenter
    """
    # get the galactocentric cylindrical coordinate
    R, phi, z, vR, vT, vz  = ICRS_to_Galactocentric_Cylindrical(ra, dec, distance, pm_ra, pm_dec, vr)
    # compute the star's eccentricity, zmax, rperi, rap
    Ecc, Zmax, Rperi, Rap = aAS.EccZmaxRperiRap(R,vR,vT,z,vz,phi)
    return  (Ecc, Zmax, Rperi, Rap)

In [258]:
def EccZmaxRperiRap_with_uncertainties (ra, dec, distance, pm_ra, pm_dec, vr, ra_u, dec_u, distance_u, pm_ra_u, pm_dec_u, vr_u):
    """
    Args:
        ra (float): the right ascension in degree
        dec (float): the declination in degree
        distance (float): the distance in kpc
        pm_ra (float): the proper motion in the right ascension direction in milliarcsec per year
        pm_dec (float): the proper motion in the declination direction in milliarcsec per year
        radial_velocity (float): the radial velocity in kilometer per second
        
        ra_u (float): uncertainty of the right ascension in degree
        dec_u (float): uncertainty of the declination in degree
        distance_u (float): uncertainty of the distance in kpc
        pm_ra_u (float): uncertainty of the proper motion in the right ascension direction in milliarcsec per year
        pm_dec_u (float): uncertainty of the proper motion in the declination direction in milliarcsec per year
        vr_u (float): uncertainty of the radial velocity in kilometer per second
        
    Returns:
        Ecc (float): the eccentricity of the orbit 
        Zmax (float): the maximum height of the orbit
        Rperi (float): the pericenter
        Rap (float): the apocenter
        
        Ecc_u (float): uncertainty of the eccentricity of the orbit 
        Zmax_u (float): uncertainty of the maximum height of the orbit
        Rperi_u (float): uncertainty of the pericenter
        Rap_u (float): uncertainty of the apocenter
        
    """
    # get the eccentricity, zmax, rperi, rap for the main star
    Ecc, Zmax, Rperi, Rap = find_EccZmaxRperiRap(ra, dec, distance, pm_ra, pm_dec, vr)
    
    # generate a Gaussian distribution centred around the star's ICRS coordinate in the 6 dimensional space
    mean = [ra, dec, distance, pm_ra, pm_dec, vr]
    # assuming the 6 uncertainties have no correlation, the covariance matrix is just a diagonal matrix with the diagonal
    # entries being the standard deviation or the given uncertainties in the coordinates
    cov = [[ra_u, 0, 0, 0, 0, 0],
           [0, dec_u, 0, 0, 0, 0],
           [0, 0, distance_u, 0, 0, 0],
           [0, 0, 0, pm_ra_u, 0, 0],
           [0, 0, 0, 0, pm_dec_u, 0],
           [0, 0, 0, 0, 0, vr_u]]
    # generate 100 stars that are normally distributed around the given star
    ra_list, dec_list, distance_list, pm_ra_list, pm_dec_list, vr_list = np.random.multivariate_normal(mean, cov, 100).T
    
    # initialize numpy array that store the list of EccZmaxRperiRap for the randomly generated stars
    Ecc_list, Zmax_list, Rperi_list, Rap_list = np.empty((4, 100))
    
    #initialize counter
    counter = 0
    # for each star in the list, calculate their eccentricity, zmax, rperi, rap
    # let the coordinate of the randomly generated star be r_ra, r_dec, etc
    for r_ra, r_dec, r_distance, r_pm_ra, r_pm_dec, r_vr in \
    zip(ra_list, dec_list, distance_list, pm_ra_list, pm_dec_list, vr_list):
        # get the EccZmaxRperiRap
        r_Ecc, r_Zmax, r_Rperi, r_Rap = find_EccZmaxRperiRap(r_ra, r_dec, r_distance, r_pm_ra, r_pm_dec, r_vr)
        # append the result to the respective list
        Ecc_list[counter] = r_Ecc
        Zmax_list[counter] = r_Zmax
        Rperi_list[counter] = r_Rperi
        Rap_list[counter] = r_vr
        
        counter = counter + 1
    
    print(Ecc_list)
    
    


In [259]:
EccZmaxRperiRap_with_uncertainties(1,1,1,1,1,1, 0.1, 0.1,0.1,0.1,0.1,0.1)

ValueError: too many values to unpack (expected 4)